<a href="https://colab.research.google.com/github/pankajkgarg/smol-course/blob/main/1_instruction_tuning/notebooks/sft_finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [2]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
# login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [10]:
# Let's test the base model before training
prompt = "Write a haiku about programming"
prompt = "What is mathematics "

print(prompt)

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

print(formatted_prompt)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
tokenizer.decode(outputs[0], skip_special_tokens=True)

What is mathematics 
<|im_start|>user
What is mathematics <|im_end|>

Before training:
user
What is mathematics 
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is mathematics petition
What is


'user\nWhat is mathematics \nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is mathematics petition\nWhat is'

## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [6]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/52.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/119 [00:00<?, ? examples/s]

In [7]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

def process_dataset(sample):
    # TODO: 🐢 Convert the sample into a chat format
    # use the tokenizer's method to apply the chat template

    result = {}
    for subset in ['train', 'test']:
      new_subset_data = []
      for message in sample[subset]:
        message_copy = message.copy()
        message_copy['messages'] = tokenizer.apply_chat_template(message_copy['messages'], add_generation_prompt=True)
        new_subset_data.append(message_copy)
      result[subset] = new_subset_data

    return sample

mod_ds = process_dataset(ds)

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [13]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=100,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-44a152f2b14d>:18: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [14]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
50,0.745700,1.132594
100,0.814200,1.116274


In [ ]:
trainer.push_to_hub(tags=finetune_tags)

In [6]:
finetune_name = "SmolLM2-FT-MyDataset"
trained_model_name = f"pankajmlai/{finetune_name}"
trained_model = AutoModelForCausalLM.from_pretrained(trained_model_name).to(device)
trained_tokenizer = AutoTokenizer.from_pretrained(trained_model_name)


dpo_finetune_name = 'SmolLM2-FT-DPO'
dpo_model_name = f"pankajmlai/{dpo_finetune_name}"
dpo_model = AutoModelForCausalLM.from_pretrained(dpo_model_name).to(device)
dpo_tokenizer = AutoTokenizer.from_pretrained(dpo_model_name)

# model_name = "HuggingFaceTB/SmolLM2-135M"
# model = AutoModelForCausalLM.from_pretrained(
#     pretrained_model_name_or_path=model_name
# ).to(device)
# tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)


config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [9]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
# prompt = "Damn what is going on?"
prompt = "What is mathematics"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = trained_tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = trained_tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
# inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = trained_model.generate(**inputs, max_new_tokens=100)


dpo_prompt = dpo_tokenizer.apply_chat_template(messages, tokenize=False)
dpo_inputs = dpo_tokenizer(formatted_prompt, return_tensors="pt").to(device)
dpo_outputs = dpo_model.generate(**dpo_inputs, max_new_tokens=100)

print("After training:")
print(trained_tokenizer.decode(outputs[0], skip_special_tokens=True))
print("DPO result:")
print(dpo_tokenizer.decode(dpo_outputs[0], skip_special_tokens=True))

#trained_tokenizer.decode(outputs[0], skip_special_tokens=True)

After training:
user
What is mathematics
What is mathematics
What is chemistry?
What is a number?
What is the therom?
What is a video?
What is a video game?
What are the terms of the economic system?
What is an individual?
What is an individual?
What is an individual?
What is an individual?
What is an individual?
What is an individual?
What is an individual?
What is an individual?
What is an individual?

DPO result:
user
What is mathematics

What is mathematics?

Mathematics is the study of numbers, shapes, and patterns. It's a way of thinking and doing that helps us understand the world around us. It's a way of expressing relationships and understanding the world in a way that's easy to understand.

What is a number?

A number is a number that has a value. For example, the number 5 is a number because it has a value of 5. It's a number that


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.